In [1]:
def split_(line):
    line = line[:-1].split(' ')  
    while '' in line:
        line.remove('')
    return line
    
regtab = {'A': 0, 'X': 1, 'L': 2, 'B': 3,'S': 4, 'T': 5, 'PC': 8, 'SW': 9}

#creating opcode table
op = open('optab')
op = op.readlines()
op = [split_(i) for i in op]
op = [ [i[0],i[2],i[3]] for i in op]
optab = {}
for i in op:
    optab[i[0]] = {'len':int(i[1]),'hex':i[2]}

#symtab
with open('symtab.txt') as f:
    sym = f.readlines()
sym = [i[:-1].split(' ') for i in sym]
symtab = {}
for i in sym:
    symtab[i[0]] = i[1]    
    

In [46]:
#tohex
def tohex(n):
    return hex((n) & 0xFFF)[2:]

In [2]:
# obj = open('obj.txt','w')
len = '000200'
prg_name = '------'
pc = 0

In [57]:
with open('temp.txt') as f:
    line = f.readline()[:-1].split(' ')
    if 'START' in line:
        if line[0] != '-':
            prg_name = line[0]        
        linr = f.readline()[:-1].split(' ')
    print(f'H^{prg_name}^{line[-1]}^{len}')
    while line:       
        flag = 0
        index = 1
        addr,op,opr = line
        if '+' in op:
            flag = 1
            op = op[1:]
        if op in optab:
            if optab[op]['len'] == 2:
                pc =+ 2
                if ',' in opr:
                    print(f"{optab[op]['hex']}{regtab[opr[0]]}{regtab[opr[-1]]}")
                else:
                    print(f"{optab[op]['hex']}{regtab[opr[0]]}0")
            elif optab[op]['len'] == 3:
                pc += 3
                if ',' in opr:      #index mode
                        index = 1
                if flag:                            # extended mode 
                    pc += 1
                    op = tohex(int(optab[op]['hex'],16)^3).zfill(2) + '1'
                    opr = symtab[opr].zfill(5)
                    print(op+opr)
                elif '#' in opr:                    #immediate
                    op = tohex(int(optab[op]['hex'],16)^1).zfill(2)
                    if (opr[1:]).isdigit:
                        opr = hex(int(opr[1:]))[2:].zfill(4)
                    print(op+opr)                          
                elif '@' in opr:                    #indirect
                    print('ooops')
                    op = tohex(int(optab[op]['hex'],16)^2).zfill(2)
                elif opr == '-':
                    op = tohex(int(optab[op]['hex'],16)^3).zfill(2)
                    print(op+'0000')
                else:
#                     if opr != '-':
                    disp = int(symtab[opr],16) - pc      
                    if disp >= -2048 and disp <= 2047:
                        d = tohex(disp).zfill(3) 
                        if index:
                              opr = 'a' + d
                        else:
                              opr = '2' + d
                    op = tohex(int(optab[op]['hex'],16)^3).zfill(2)  
                    print(op+opr)
                    
        line = f.readline()[:-1].split(' ')
        if 'END' in line:
            break
            
    

H^FIBO^00000^000200
6d000a
4b100011
4b10002e
e3a027
33a00b
dba021
A005
3ba00e
A004
37a00e
0fa006
4f0000
050000
6d0001
010000
77aff7
e3a03c
33a01d
dfa036
9040
0fa053
7fa053
03a050
6fa04a
3ba035
